In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow_decision_forests --upgrade

In [ ]:
!pip install tensorflow==2.5.0

In [ ]:
!pip install -q wurlitzer

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
train.head()

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
test.head()

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')
sample_submission.head()

In [ ]:
train.drop('id', axis=1, inplace = True)
test.drop('id', axis=1, inplace = True)

In [ ]:
train_copy = train.copy()

In [ ]:
train.drop('target', axis=1, inplace = True)

In [ ]:
data = pd.concat([train,test], axis=0)
data.head()

In [ ]:
# def make_numerical_transformation(data):
#     np.seterr(divide = 'ignore')
#     for i in data.columns:
#         data[i] = np.log(data[i]-(min(data[i]-1)))
# make_numerical_transformation(data)

In [ ]:
# data.head()

In [ ]:
train = data[:100000]
test = data[100000:]

In [ ]:
train = pd.concat([train,train_copy['target']], axis=1)
train.head()

In [ ]:
train.replace({'Class_1': 0, 'Class_2': 1, 'Class_3':2, 'Class_4': 3}, inplace = True)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train_df = train[0:80000]
eval_df = train[80000:]

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label = 'target')

In [ ]:
eval_ds = tfdf.keras.pd_dataframe_to_tf_dataset(eval_df, label = 'target')

In [ ]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label = None)

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(num_trees=100, growing_strategy="BEST_FIRST_GLOBAL", max_depth=8,hyperparameter_template="benchmark_rank1")

In [ ]:
model.compile(metrics=["accuracy"])

In [ ]:
model.fit(train_ds)

In [ ]:
model.evaluate(eval_ds)

In [ ]:
model.summary()

In [ ]:
model.save('first_model')

In [ ]:
model.make_inspector().training_logs()

In [ ]:
import matplotlib.pyplot as plt

logs = model.make_inspector().training_logs()

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

plt.subplot(1, 2, 2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Logloss (out-of-bag)")

plt.show()

In [ ]:
#%load_ext tensorboard

In [ ]:
#model.make_inspector().export_to_tensorboard("/tmp/tensorboard_logs")

In [ ]:
#%tensorboard --logdir "/tmp/tensorboard_logs"

In [ ]:
y_pred = model.predict(test_ds)

In [ ]:
sample_submission.iloc[:,1:5] = y_pred
sample_submission

In [ ]:
sample_submission.to_csv('submission_df.csv',index=False)